### **Importing Libraries**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset
from torch.utils.data import Dataset
import torchvision.models as models
from sklearn.preprocessing import RobustScaler
from torchvision import transforms
import torchvision
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torchmetrics.segmentation import MeanIoU
import torchvision.transforms as T
from PIL import Image
import math
from sklearn.preprocessing import MinMaxScaler

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


### **Dataset**

#### **Helper Functions**

In [2]:
transform_base = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

transform_color = T.Compose([
    T.Resize((256, 256)),
    T.ColorJitter(brightness=0.3, contrast=0.2, saturation=0.2, hue=0.1),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_affine = T.Compose([
    T.Resize((256, 256)),
    T.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=(0.8, 1.2), shear=0.2),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_val = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def angle_to_vector(theta_deg):
    theta_rad = math.radians(theta_deg)
    return torch.tensor([math.cos(theta_rad), math.sin(theta_rad)], dtype=torch.float32)

def vector_to_angle(vector):
    cos_theta, sin_theta = vector
    angle_rad = torch.atan2(sin_theta, cos_theta)
    angle_deg = angle_rad * (180 / math.pi)
    # Ensure angle is in [0, 360) range
    return (angle_deg + 360) % 360

#### **Dataset Class**

In [3]:
class LatitudeDataset(Dataset):
    def __init__(self, image_dir, labels_df, transform=None, scaler=None, fit_scaler=True, log_transform=True):
        self.image_dir = image_dir
        self.labels_df = labels_df
        self.transform = transform
        self.log_transform = log_transform
        
        latitudes = labels_df['latitude'].values.reshape(-1, 1)
        
        # Log transform to manage wide range of values like 234876
        if self.log_transform:
            transformed_latitudes = np.sign(latitudes) * np.log1p(np.abs(latitudes))
        else:
            transformed_latitudes = latitudes
            
        # Use RobustScaler to handle outliers
        if scaler is None:
            self.scaler = RobustScaler()
            if fit_scaler:
                self.scaler.fit(transformed_latitudes)
        else:
            self.scaler = scaler
            
        self.scaled_latitudes = self.scaler.transform(transformed_latitudes).flatten()
        self.original_latitudes = latitudes.flatten()

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        row = self.labels_df.iloc[idx]
        img_path = os.path.join(self.image_dir, row['filename'])
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        scaled_latitude = torch.tensor(self.scaled_latitudes[idx], dtype=torch.float32)
        
        return image, scaled_latitude
        
    def inverse_transform(self, scaled_values):
        if torch.is_tensor(scaled_values):
            scaled_values = scaled_values.cpu().numpy()
            
        if scaled_values.ndim == 1:
            scaled_values = scaled_values.reshape(-1, 1)
            
        transformed_values = self.scaler.inverse_transform(scaled_values)
        
        # Reverse the log1p transform
        if self.log_transform:
            original_values = np.sign(transformed_values) * (np.expm1(np.abs(transformed_values)))
        else:
            original_values = transformed_values
            
        return original_values.flatten()


#### **Extend Dataset**

In [4]:
def create_extended_dataset(image_dir, labels_df):
    # Original dataset
    original_dataset = LatitudeDataset(
        image_dir=image_dir,
        labels_df=labels_df,
        transform=transform_base,
        log_transform=True,
    )
    
    # Color jitter augmented dataset
    color_dataset = LatitudeDataset(
        image_dir=image_dir,
        labels_df=labels_df,
        transform=transform_color,
        log_transform=True
    )
    
    # # Affine transform augmented dataset
    affine_dataset = LatitudeDataset(
        image_dir=image_dir,
        labels_df=labels_df,
        transform=transform_affine,
        log_transform=True
    )
    
    extended_dataset = ConcatDataset([original_dataset, color_dataset, affine_dataset])
    
    return original_dataset

#### **Training**

In [5]:
image_dir_train = "Dataset/Train/images_train"
labels_path_train = "Dataset/Train/labels_train.csv"

labels_df = pd.read_csv(labels_path_train)

train_dataset = create_extended_dataset(image_dir_train, labels_df)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

#### **Validation**

In [6]:
images_dir_val = "Dataset/Val/images_val"
labels_path_val = "Dataset/Val/labels_val.csv"
labels_df_val = pd.read_csv(labels_path_val)

val_dataset = LatitudeDataset(
    image_dir=images_dir_val,
    labels_df=labels_df_val,
    transform=transform_val,
    scaler=train_dataset.scaler,
    fit_scaler=False,
    log_transform=True
)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

### **Model**

#### **Model Implementation**

In [7]:
class LatitudeRegressor(nn.Module):
    def __init__(self, pretrained=True, dropout_rate=0.3):
        super().__init__()
        base_model = models.resnet50(weights='DEFAULT' if pretrained else None)
        num_features = base_model.fc.in_features
        self.features = nn.Sequential(*list(base_model.children())[:-1])
        
        # Regression head for latitude prediction
        self.regressor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_features, 1024),
            nn.BatchNorm1d(1024), 
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            
            nn.Linear(512, 1)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.regressor(x)
        return x

### **Training**

In [8]:
def train_regression_model(model, train_loader, val_loader, optimizer, num_epochs, device, loss_fn):
   model.to(device)
   best_val_loss = float('inf')

   for epoch in range(num_epochs):
       model.train()
       train_loss = 0.0
       pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")

       for images, targets in pbar:
           images = images.to(device)
           targets = targets.to(device)

           preds = model(images)
           loss = loss_fn(preds.squeeze(), targets)

           optimizer.zero_grad()
           loss.backward()
           optimizer.step()

           train_loss += loss.item() * images.size(0)
           pbar.set_postfix(loss=loss.item())

       avg_train_loss = train_loss / len(train_loader.dataset)

       # Validation loop
       model.eval()
       val_loss = 0.0
       all_preds = []
       all_targets = []
       
       with torch.no_grad():
           pbar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]")
           for images, targets in pbar:
               images = images.to(device)
               targets = targets.to(device)

               preds = model(images)
               loss = loss_fn(preds.squeeze(), targets)

               val_loss += loss.item() * images.size(0)
               pbar.set_postfix(loss=loss.item())


       avg_val_loss = val_loss / len(val_loader.dataset)
       if avg_val_loss < best_val_loss:
           best_val_loss = avg_val_loss
           torch.save(model.state_dict(), 'best_model.pth')

       print(f"Epoch {epoch+1}: Train MSE = {avg_train_loss:.4f}, Val MSE = {avg_val_loss:.4f}")

In [9]:
def evaluate_model(model, test_loader, device, dataset=None):
    
    model.eval()
    scaled_predictions = []
    scaled_ground_truths = []
    
    with torch.no_grad():
        for inputs, scaled_latitudes in tqdm(test_loader, desc="Evaluating"):
            inputs = inputs.to(device)
            scaled_latitudes = scaled_latitudes.to(device)
            
            outputs = model(inputs)
            
            scaled_predictions.extend(outputs.cpu().numpy())
            scaled_ground_truths.extend(scaled_latitudes.cpu().numpy())
    
    scaled_predictions = np.array(scaled_predictions).flatten()
    scaled_ground_truths = np.array(scaled_ground_truths).flatten()
    
    scaled_mse = np.mean((scaled_predictions - scaled_ground_truths) ** 2)
    scaled_rmse = np.sqrt(scaled_mse)
    scaled_mae = np.mean(np.abs(scaled_predictions - scaled_ground_truths))
    
    results = {
        'scaled_mse': scaled_mse,
        'scaled_rmse': scaled_rmse,
        'scaled_mae': scaled_mae,
    }
    
    if dataset is not None:
        unscaled_predictions = dataset.inverse_transform(scaled_predictions)
        unscaled_ground_truths = dataset.inverse_transform(scaled_ground_truths)
        
        unscaled_mse = np.mean((unscaled_predictions - unscaled_ground_truths) ** 2)
        unscaled_rmse = np.sqrt(unscaled_mse)
        
        results.update({
            'unscaled_mse': unscaled_mse,
            'unscaled_rmse': unscaled_rmse,
            'predictions': unscaled_predictions,
            'ground_truths': unscaled_ground_truths
        })
        
        errors = np.abs(unscaled_predictions - unscaled_ground_truths)
        percentiles = [50, 75, 90, 95, 99]
        error_percentiles = np.percentile(errors, percentiles)
        
        for i, p in enumerate(percentiles):
            results[f'p{p}_error'] = error_percentiles[i]
    
    return results

In [10]:
model = LatitudeRegressor()
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.05)
train_regression_model(model, train_loader, val_loader, optimizer, num_epochs=50, device=device, loss_fn=loss_fn)


Epoch 1/50 [Val]:  96%|█████████▌| 23/24 [00:03<00:00,  5.62it/s, loss=18.6]   /home/chetan/miniconda3/envs/localEnv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 1/50 [Val]: 100%|██████████| 24/24 [00:03<00:00,  7.60it/s, loss=9.16]


Epoch 1: Train MSE = 3354.8151, Val MSE = 675.2851


Epoch 2/50 [Val]: 100%|██████████| 24/24 [00:03<00:00,  7.51it/s, loss=5.9]    


Epoch 2: Train MSE = 3353.1985, Val MSE = 587.4825


Epoch 3/50 [Val]: 100%|██████████| 24/24 [00:03<00:00,  7.51it/s, loss=3.37]   


...

Epoch 48: Train MSE = 2759.0069, Val MSE = 601.5069


Epoch 49/50 [Val]: 100%|██████████| 24/24 [00:02<00:00,  8.39it/s, loss=115]    


Epoch 49: Train MSE = 2836.8547, Val MSE = 584.0459


Epoch 50/50 [Val]: 100%|██████████| 24/24 [00:02<00:00,  8.43it/s, loss=39.2]   

Epoch 50: Train MSE = 2832.9895, Val MSE = 567.9119


In [11]:
# Evaluate the model on the validation set
results = evaluate_model(model, val_loader, device)
print("Scaled MSE:", results['scaled_mse'])
print("Unscaled MSE:", results['unscaled_mse'])

Evaluating: 100%|██████████| 24/24 [00:02<00:00,  8.37it/s]

Scaled MSE: 567.9119


KeyError: 'unscaled_mse'

In [ ]:
# Load the best model for evaluation

model_ev = LatitudeRegressor()
model_ev.to(device)
model_ev.load_state_dict(torch.load('best_model.pth'))

# Evaluate the model on the validation set
results = evaluate_model(model_ev, val_loader, device)
print(f"Mean Angle Error: {results['mean_angle_error']:.4f} degrees")
print(f"Median Angle Error: {results['median_angle_error']:.4f} degrees")
print(f"MSE (sin/cos): {results['mse_sin_cos']:.4f}")

Mean Angle Error: 30.8790 degrees
Median Angle Error: 19.3219 degrees
MSE (sin/cos): 0.4744
